In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider, fixed
import ipywidgets as widgets

def draw_sphere(ax, center, radius=50):
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
    y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
    z = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, color='b', alpha=0.3)

def draw_axes(ax, center, matrix, length=50):
    axes = np.array([
        [length, 0, 0, 1],
        [0, length, 0, 1],
        [0, 0, length, 1]
    ])
    transformed_axes = np.dot(matrix, axes.T).T
    for i, color in enumerate(['r', 'g', 'b']):
        ax.quiver(center[0], center[1], center[2],
                  transformed_axes[i, 0]-center[0], transformed_axes[i, 1]-center[1], transformed_axes[i, 2]-center[2],
                  color=color, arrow_length_ratio=0.1)

def update_matrix(rotation_x, rotation_y, rotation_z, translation_x, translation_y, translation_z):
    Rx = np.array([
        [1, 0, 0, 0],
        [0, np.cos(np.radians(rotation_x)), -np.sin(np.radians(rotation_x)), 0],
        [0, np.sin(np.radians(rotation_x)), np.cos(np.radians(rotation_x)), 0],
        [0, 0, 0, 1]
    ])
    Ry = np.array([
        [np.cos(np.radians(rotation_y)), 0, np.sin(np.radians(rotation_y)), 0],
        [0, 1, 0, 0],
        [-np.sin(np.radians(rotation_y)), 0, np.cos(np.radians(rotation_y)), 0],
        [0, 0, 0, 1]
    ])
    Rz = np.array([
        [np.cos(np.radians(rotation_z)), -np.sin(np.radians(rotation_z)), 0, 0],
        [np.sin(np.radians(rotation_z)), np.cos(np.radians(rotation_z)), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    T = np.array([
        [1, 0, 0, translation_x],
        [0, 1, 0, translation_y],
        [0, 0, 1, translation_z],
        [0, 0, 0, 1]
    ])
    return np.dot(T, np.dot(Rz, np.dot(Ry, Rx)))

def plot_3d(matrix):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    plt.close(fig)
    
    original_center = np.array([0, 0, 0, 1])
    transformed_center = np.dot(matrix, original_center)

    draw_sphere(ax, original_center[:3])
    draw_axes(ax, original_center[:3], np.eye(4))
    draw_sphere(ax, transformed_center[:3])
    draw_axes(ax, transformed_center[:3], matrix)

    # Set the plot limits
    ax.set_xlim([-200, 200])
    ax.set_ylim([-200, 200])
    ax.set_zlim([-200, 200])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    return fig

def interactive_transformations(rotation_x=0, rotation_y=0, rotation_z=0, translation_x=0, translation_y=0, translation_z=0):
    matrix = update_matrix(rotation_x, rotation_y, rotation_z, translation_x, translation_y, translation_z)
    display(plot_3d(matrix))
    print("Transformation Matrix:")
    print(matrix)

sliders = {
    'rotation_x': FloatSlider(min=-180, max=180, step=1, value=0, description='Rotate X (deg)'),
    'rotation_y': FloatSlider(min=-180, max=180, step=1, value=0, description='Rotate Y (deg)'),
    'rotation_z': FloatSlider(min=-180, max=180, step=1, value=0, description='Rotate Z (deg)'),
    'translation_x': FloatSlider(min=-100, max=100, step=1, value=0, description='Translate X'),
    'translation_y': FloatSlider(min=-100, max=100, step=1, value=0, description='Translate Y'),
    'translation_z': FloatSlider(min=-100, max=100, step=1, value=0, description='Translate Z')
}

ui = widgets.HBox([
    widgets.VBox([sliders['rotation_x'], sliders['rotation_y'], sliders['rotation_z']]),
    widgets.VBox([sliders['translation_x'], sliders['translation_y'], sliders['translation_z']])
])

out = widgets.interactive_output(interactive_transformations, sliders)

display(ui, out)


Output()